## 0. Import

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

import models

from mmengine.config import Config
from mmengine.runner import Runner
from mmengine.structures import InstanceData
from mmdet3d.structures.bbox_3d.lidar_box3d import LiDARInstance3DBoxes

## 1. Prepare Dataloader/Metric/BaseModel from MMDetection3D

In [ ]:
config     = "mmdetection3d/projects/BEVFusion/configs/bevfusion_lidar-cam_voxel0075_second_secfpn_8xb4-cyclic-20e_nus-3d.py"
cfg = Config.fromfile(config)
cfg.work_dir = 'mmdetection3d/work_dirs/onnx'

runner = Runner.from_cfg(cfg)

In [ ]:
dataloader = runner.val_dataloader
evaluator = runner.val_evaluator

## 2. Define Custom Model


In [ ]:
batch_size = dataloader.batch_size
device = "cuda"

# model = models.BEVFusionOnnxRuntimeModel(batch_size=batch_size, device=device, is_simplified=True)
# model = models.BEVFusionTensorRTModel(batch_size=batch_size, device=device)
model = models.BEVFusionTVMModel()

## 3. Inference

In [ ]:
# nuscenes 기본 normalization

mean = torch.tensor([123.675, 116.280, 103.530], device=device).view(3,1,1)
std  = torch.tensor([ 58.395,  57.120,  57.375], device=device).view(3,1,1)

In [ ]:
bboxes_3ds = list()
scores_3ds = list()
labels_3ds = list()

pbar = tqdm(dataloader, desc='Val')
for e, data in enumerate(pbar):

    imgs = torch.stack(data["inputs"]["img"]).to(device)
    imgs = (imgs - mean) / std

    points = [p.to(device) for p in data["inputs"]["points"]]
    img_aug_matrix = np.stack([np.stack(sample.img_aug_matrix) for sample in data["data_samples"]])
    lidar_aug_matrix = np.stack([np.eye(4) for  _ in range(len(points))])
    cam2img = np.stack([sample.cam2img for sample in data["data_samples"]])
    cam2lidar = np.stack([sample.cam2lidar for sample in data["data_samples"]])
    lidar2img = np.stack([sample.lidar2img for sample in data["data_samples"]])

    metas = {
        "img_aug_matrix":   torch.from_numpy(img_aug_matrix.astype(np.float32)).to(device),
        "lidar_aug_matrix": torch.from_numpy(lidar_aug_matrix.astype(np.float32)).to(device),
        "cam2img":          torch.from_numpy(cam2img.astype(np.float32)).to(device),
        "cam2lidar":        torch.from_numpy(cam2lidar.astype(np.float32)).to(device),
        "lidar2img":        torch.from_numpy(lidar2img.astype(np.float32)).to(device),
    }
    
    bboxes_3d, scores_3d, labels_3d = model(imgs, points, metas)
    for B in range(len(points)):
        bboxes_3ds.append(LiDARInstance3DBoxes(bboxes_3d[B], box_dim=9, origin=(0.5, 0.5, 0.5))),
        scores_3ds.append(scores_3d[B]),
        labels_3ds.append(labels_3d[B])

    postfix = {key : f"{value:.1f}ms" for key, value in model.get_avg_latencies().items()}
    pbar.set_postfix(postfix)

## 4. Evaluation

In [ ]:
for i in range(len(dataloader.dataset)):
    data_sample = {
        'pred_instances_3d': InstanceData(metainfo={
            'bboxes_3d': bboxes_3ds[i].cpu(),
            'scores_3d': scores_3ds[i].cpu(),
            'labels_3d': labels_3ds[i].cpu(),
        }),
        'pred_instances': InstanceData(),
        'sample_idx': i
    }
    evaluator.process(data_samples=[data_sample])

In [ ]:
results = evaluator.evaluate(len(dataloader.dataset))